# Cluster/local configs

In [1]:
# If on cluster, cwd is '/rds/general/user/nz423', not the project directory, breaking imports
import os
import sys
from pathlib import Path

project_dir = Path(os.getcwd()) / "exploring-hydra-boosting"
if "rds" not in project_dir.parts:
    project_dir = project_dir.parent
    datasets_dir = project_dir.parent.parent / "Data"
else:
    datasets_dir = project_dir / "DATASETS"
    if str(project_dir) not in sys.path:
        sys.path.append(str(project_dir))
    
class Config:
    project_dir = project_dir
    TSER_dir = datasets_dir / "TSER"
    BigTSC_dir = datasets_dir / "BigTSC"
    UTSC_dir = datasets_dir / "UTSC"
    results_dir = project_dir / "results_TESTING"

print(Config.project_dir)
print(Config.TSER_dir)
print(Config.BigTSC_dir)
print(Config.UTSC_dir)
print(Config.results_dir)

/rds/general/user/nz423/home/exploring-hydra-boosting
/rds/general/user/nz423/home/exploring-hydra-boosting/DATASETS/TSER
/rds/general/user/nz423/home/exploring-hydra-boosting/DATASETS/BigTSC
/rds/general/user/nz423/home/exploring-hydra-boosting/DATASETS/UTSC
/rds/general/user/nz423/home/exploring-hydra-boosting/results_TESTING


# Simple Wrapper no gridsearch

In [2]:
# class TSMLWrapperHydraBoost(RegressorMixin, BaseTimeSeriesEstimator):
    
#     def __init__(self, **kwargs):
#         super(TSMLWrapperHydraBoost, self).__init__()
#         self.hydraboost = HydraBoost(
#             n_layers=1,
#             init_n_kernels=8,
#             init_n_groups=64,
#             n_kernels=8,
#             n_groups=64,
#             max_num_channels=3,
#             hydra_batch_size=10000,
#             l2_reg=10,
#             l2_ghat=0.1,
#             boost_lr=1,
#             train_top_at = [0, 5, 10],
#             **kwargs
#         )
        

#     def fit(self, X: np.ndarray, y: np.ndarray) -> object:
#         """Fit the estimator to training data.

#         Parameters
#         ----------
#         X : 3D np.ndarray of shape (n_instances, n_channels, n_timepoints)
#             The training data.
#         y : 1D np.ndarray of shape (n_instances)
#             The target labels for fitting, indices correspond to instance indices in X

#         Returns
#         -------
#         self :
#             Reference to self.
#         """
#         X = torch.from_numpy(X).float()
#         y = torch.from_numpy(y).float()
#         y = y.unsqueeze(1)
#         self.X_mean = X.mean()
#         self.X_std = X.std()
#         self.y_mean = y.mean()
#         self.y_std = y.std()
#         X = (X - self.X_mean) / self.X_std
#         y = (y - self.y_mean) / self.y_std
#         self.hydraboost.fit(X, y)
#         return self


#     def predict(self, X: np.ndarray) -> np.ndarray:
#         """Predicts labels for sequences in X.

#         Parameters
#         ----------
#         X : 3D np.ndarray of shape (n_instances, n_channels, n_timepoints)
#             The training data.

#         Returns
#         -------
#         y : array-like of shape (n_instances)
#             Predicted target labels.
#         """
#         X = torch.from_numpy(X).float()
#         X = (X - self.X_mean) / self.X_std
#         pred = self.hydraboost(X)
#         pred = pred * self.y_std + self.y_mean
#         return pred.squeeze().detach().numpy()
        
        

#     def _more_tags(self) -> dict:
#         return {
#             "X_types": ["3darray"],
#             "equal_length_only": True,
#             "allow_nan": False,
#         }

In [3]:
# test_regressor(
#     regressor = TSMLWrapperHydraBoost(),
#     regressor_name = "HydraBoost",
# )

# Gridsearch

In [4]:
from run_regression_experiments import test_regressor, TSMLGridSearchWrapper
from models.random_feature_representation_boosting import HydraFeatureBoost
from models.label_space_boost import HydraLabelBoost


# test_regressor(
#     regressor_name = "HydraBoostGridSearchHoldout",
#     regressor = TSMLGridSearchWrapper(
#                 "holdout",
#                 seed=0,
#                 device="cuda",
#                 modelClass=HydraLabelBoost,
#                 model_param_grid={
#                     "n_estimators": [2],
#                     "n_kernels": [8],
#                     "n_groups": [64],
#                     "hydra_batch_size": [10000],
#                     "l2_reg": [10],
#                     "boost_lr": [1.0],
#                     "find_amount_of_say": [True],
#                 },
#                 ),
#     dataset_name = "HouseholdPowerConsumption1",
#     TSER_data_dir = Config.TSER_dir,
#     results_dir = Config.project_dir / "results_TESTING",
#     resample_id=0,
# )


/rds/general/user/nz423/home/miniforge3/envs/hydraboost/lib/python3.11/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(
/rds/general/user/nz423/home/miniforge3/envs/hydraboost/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
%run {Config.project_dir / "run_regression_experiments.py"} \
    --models "HydraLabelReuseBoost" \
    --resample_ids 0 \
    --results_dir {Config.results_dir.as_posix()} \
    --TSER_dir {Config.TSER_dir.as_posix()} \
    --device "cuda" \
    --optuna_or_gridsearch "optuna" \
    --holdout_or_kfold "holdout" \
    --seed 0 \
    --dataset_indices 50 

# commented to run all datasets, also dont forget to readd \

/rds/general/user/nz423/home/exploring-hydra-boosting/DATASETS/TSER
/rds/general/user/nz423/home/exploring-hydra-boosting/results_TESTING


[I 2025-02-24 18:08:00,148] A new study created in memory with name: no-name-22077a99-868f-41a1-90b6-6ecdf4a3a201
[I 2025-02-24 18:08:00,376] Trial 0 finished with value: 0.767451286315918 and parameters: {'n_estimators': 6, 'l2_reg': 72.57005721594277, 'boost_lr': 0.6424870384644795}. Best is trial 0 with value: 0.767451286315918.
[I 2025-02-24 18:08:00,585] Trial 1 finished with value: 0.7657219767570496 and parameters: {'n_estimators': 6, 'l2_reg': 4.950159553733191, 'boost_lr': 0.6813047017599905}. Best is trial 1 with value: 0.7657219767570496.
[I 2025-02-24 18:08:00,762] Trial 2 finished with value: 0.8046567440032959 and parameters: {'n_estimators': 5, 'l2_reg': 369.05577292137576, 'boost_lr': 0.9672964844509263}. Best is trial 1 with value: 0.7657219767570496.
[I 2025-02-24 18:08:00,905] Trial 3 finished with value: 0.7992616891860962 and parameters: {'n_estimators': 4, 'l2_reg': 146.85885989200833, 'boost_lr': 0.5760054277776141}. Best is trial 1 with value: 0.7657219767570496

self.best_params {'n_estimators': 10, 'l2_reg': 99.34607487106175, 'boost_lr': 0.9701898513844643}
16.869077641416 mse
4.107198271500415 rmse
0.03629785134618477 mape
0.585477761719801 r2
24837.0 fit time


/rds/general/user/nz423/home/miniforge3/envs/hydraboost/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [14]:
%run {Config.project_dir / "run_regression_experiments.py"} \
    --models "HydraLabelReuseBoost" \
    --resample_ids 0 \
    --results_dir {Config.results_dir.as_posix()} \
    --TSER_dir {Config.TSER_dir.as_posix()} \
    --device "cuda" \
    --optuna_or_gridsearch "optuna" \
    --holdout_or_kfold "holdout" \
    --seed 0 \
    --dataset_indices 50 


/rds/general/user/nz423/home/exploring-hydra-boosting/DATASETS/TSER
/rds/general/user/nz423/home/exploring-hydra-boosting/results_TESTING


[I 2025-02-24 18:13:24,185] A new study created in memory with name: no-name-2b595dac-3dcd-48a1-a03c-d61e5dd64ccf
[I 2025-02-24 18:13:24,413] Trial 0 finished with value: 0.767451286315918 and parameters: {'n_estimators': 6, 'l2_reg': 72.57005721594277, 'boost_lr': 0.6424870384644795}. Best is trial 0 with value: 0.767451286315918.
[I 2025-02-24 18:13:24,622] Trial 1 finished with value: 0.7657219171524048 and parameters: {'n_estimators': 6, 'l2_reg': 4.950159553733191, 'boost_lr': 0.6813047017599905}. Best is trial 1 with value: 0.7657219171524048.
[I 2025-02-24 18:13:24,798] Trial 2 finished with value: 0.8046567440032959 and parameters: {'n_estimators': 5, 'l2_reg': 369.05577292137576, 'boost_lr': 0.9672964844509263}. Best is trial 1 with value: 0.7657219171524048.
[I 2025-02-24 18:13:24,940] Trial 3 finished with value: 0.7992615699768066 and parameters: {'n_estimators': 4, 'l2_reg': 146.85885989200833, 'boost_lr': 0.5760054277776141}. Best is trial 1 with value: 0.7657219171524048

self.best_params {'n_estimators': 10, 'l2_reg': 99.34607487106175, 'boost_lr': 0.9701898513844643}
16.869079423408614 mse
4.1071984884357144 rmse
0.036297854029085586 mape
0.5854777179310663 r2
24821.0 fit time


/rds/general/user/nz423/home/miniforge3/envs/hydraboost/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
